## __Speech → Transcript → Call notes__

This Jupyter notebook aims at transforming my meetings' recordings (audio) into calls notes. 

__→ All you need is: an iPhone (Recording), a Mac (to run this Notebook), working API keys.__

The steps are: 

1. get the transcript from the audio using [Whisper](https://openai.com/index/whisper/) (via [GroqCloud](https://console.groq.com/docs/speech-to-text)) 

2. create a note call from the transcript using [OpenAI](http://openai.com/)

In [ ]:
import os
import io
import time
from pathlib import Path
from dotenv import load_dotenv
from pydub import AudioSegment
from groq import Groq, RateLimitError, InternalServerError

# ── CONFIG ──
load_dotenv()
client = Groq(api_key=os.environ["GROQ_API_KEY"])

DOWNLOADS = Path.home() / "Downloads"
CHUNK_MS   = 60 * 1000   # 60-second chunks
PAUSE_SEC  = 3           # wait 3 seconds between each API call
MAX_RETRIES = 3          # retry up to 3 times on 429 or 503
BACKOFF_BASE = 3         # base backoff seconds for retries (exponential)

for audio_path in DOWNLOADS.glob("*.m4a"):
    print(f"\n→ Loading “{audio_path.name}”…")
    audio = AudioSegment.from_file(audio_path)
    chunks = [audio[i : i + CHUNK_MS] for i in range(0, len(audio), CHUNK_MS)]
    all_text = []

    for idx, chunk in enumerate(chunks, start=1):
        buf = io.BytesIO()
        chunk.export(buf, format="mp3")
        buf.seek(0)

        # Try up to MAX_RETRIES times if we hit 429 or 503
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = client.audio.transcriptions.create(
                    file=(f"{audio_path.stem}_part{idx}.mp3", buf.read()),
                    model="whisper-large-v3",
                    response_format="verbose_json",
                    timeout=120,
                )
                all_text.append(resp.text.strip())
                break  # success → exit retry loop

            except RateLimitError:
                wait = BACKOFF_BASE * attempt
                print(f"429 rate_limit_exceeded (chunk {idx}), retry {attempt}/{MAX_RETRIES} in {wait}s…")
                time.sleep(wait)
                buf.seek(0)
                continue

            except InternalServerError:
                wait = BACKOFF_BASE * attempt
                print(f"503 service_unavailable (chunk {idx}), retry {attempt}/{MAX_RETRIES} in {wait}s…")
                time.sleep(wait)
                buf.seek(0)
                continue

        else:
            # If we exhausted retries, raise an error
            raise RuntimeError(f"Failed to transcribe chunk {idx} after {MAX_RETRIES} attempts.")

        # Regardless of success or after retrying, wait PAUSE_SEC before the next call
        print(f"  → Waiting {PAUSE_SEC}s to respect 20 req/min…")
        time.sleep(PAUSE_SEC)

    # Stitch all chunk transcripts together
    full_transcript = "\n\n".join(all_text)
    out_path = audio_path.with_suffix(".txt")
    out_path.write_text(full_transcript, encoding="utf-8")
    print(f"Saved transcript to {out_path}")


In [ ]:
# TRANSCRIPTION -> NOTES
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

for transcript_path in DOWNLOADS.glob("*.txt"):
    print(f"\n→ Processing {transcript_path.name}...")
    
    transcript = transcript_path.read_text(encoding="utf-8")

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "user", 
            "content": (
                """
                You are an assistant whose job is to convert a full meeting transcript into concise, well-structured call notes. Follow these instructions exactly:
                1. Extract and list **Attendees** (with roles or companies, if available) at the top.
                2. Write a brief **Overview** (2–3 sentences) summarizing the meeting’s purpose.
                3. Identify all major **Topics** discussed, then group related points under each topic—even if they appeared at different times in the transcript. For each topic:
                - Use a clear heading (e.g., “Topic: Compliance & Frameworks”).
                - Present bullet points covering key facts, explanations, and names/figures mentioned.
                4. Under a separate **Decisions** heading, list every decision made (e.g., “Start SOC 2 Type 1 audit by end of quarter,” “Engage Assurance Lab as auditor”).
                5. Under **Action Items**, list each task, assigning it to a person or team when possible and noting any deadlines (e.g., “Victor to send detailed pricing breakdown by June 6,” “Client to discuss evaluation feedback with co-founders by next week”)."
                6. Include all relevant names, concepts, and figures (e.g., “SOC 2 Type 1,” “$12 000/year for the Foundation package,” “Auditor #1 fee ~$6 000”), and do not omit any important detail.
                7. Keep bullets short (no more than two sentences each) and order them logically under each heading.
                8. Maintain consistent formatting:  
                - Headings in bold (e.g., **Topic: Audit Preparation & Support**).  
                - Sub-bullets only if needed for clarification.
                9. If the same issue was raised multiple times, combine those references under a single bullet and note that it recurred.
                10. Write in neutral, business-formal tone, as if these notes will be shared with senior leadership.
                """

            )
            },
            {"role": "user",
            "content": f"Here is the transcript of a meeting:\n\n{transcript}\n\n"}

        ],
        temperature=0.2,
        max_tokens=1500,
    )
    
    summary = response.choices[0].message.content.strip()
    out_path = transcript_path.with_suffix(".summary.txt")
    out_path.write_text(summary, encoding="utf-8")

    print(f"Summared notes saved to {out_path.name}")